# Mini Project -2 

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import requests
import sqlite3
import lxml.html
import plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.offline.init_notebook_mode(connected=True)

Review Focus Item 09: Web scraping to SQL

    Find a website which contains some data you'd be interested in loading.

    Create a sqlite database with at least one table to structure the source data you are interested in capturing.

    Write a code which downloads the data from the website into the sqlite table

    Demonstrate the presence of data in the sqlite table with a SELECT statement

#Selected Website is: https://top100.winespectator.com/lists/. This site provides a list of Top 100 wines based on quality, value, availability and excitement. It lists the following information for each entry: Rank, Wine Description, Vintage, Score and Price.

In [2]:
r = requests.get("https://top100.winespectator.com/lists/")

In [3]:
r

<Response [200]>

#We can get the entire html source code by runnung r.text

In [4]:
r.text

'<!DOCTYPE html>\n<html lang="en">\n\n    <head>\n    <meta charset="utf-8">\n    <meta http-equiv="x-ua-compatible" content="ie=edge">\n\n    <title>\n        All Lists of Top 100 Wines                    | Wine Spectator\'s Top 100\n            </title>\n\n    <meta name="description" content="Each year, Wine Spectator editors select the most exciting wines we\'ve reviewed for our Top 100 winesâ\x80\x94based on quality, value, availability and an X-factor. Our archives contain every Top 100 list from 2018 back to the debut year, 1988, with full reviews, including rank, score, release price, tasting note and recommended timeframe to drink.">\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n\n    <meta property="og:title" content="Wine Spectator\'s Top 100 Winesâ\x80\x94All Lists">\n    <meta property="og:type" content="article">\n    <meta property="og:description" content="Each year, Wine Spectator editors select the most exciting wines they\'ve reviewed for

#HTML code has hierarchy - Parent & Child elements. New variable html can be created to sequence all rows in table

In [5]:
sample = lxml.html.fromstring(r.text)

In [6]:
help(sample)

Help on HtmlElement in module lxml.html object:

class HtmlElement(lxml.etree.ElementBase, HtmlMixin)
 |  ElementBase(*children, attrib=None, nsmap=None, **_extra)
 |  
 |  The public Element class.  All custom Element classes must inherit
 |  from this one.  To create an Element, use the `Element()` factory.
 |  
 |  BIG FAT WARNING: Subclasses *must not* override __init__ or
 |  __new__ as it is absolutely undefined when these objects will be
 |  created or destroyed.  All persistent state of Elements must be
 |  stored in the underlying XML.  If you really need to initialize
 |  the object after creation, you can implement an ``_init(self)``
 |  method that will be called directly after object creation.
 |  
 |  Subclasses of this class can be instantiated to create a new
 |  Element.  By default, the tag name will be the class name and the
 |  namespace will be empty.  You can modify this with the following
 |  class attributes:
 |  
 |  * TAG - the tag name, possibly containing a 

In [7]:
sample

<Element html at 0x26d3f7ea4a8>

#In the website, the table we are looking for has a table id = table

In [8]:
list_table = sample.get_element_by_id("table")

#Lets check what is in the list_table- It is a table with elements.

In [9]:
list_table

<Element table at 0x26d3f7b7a48>

#Lets get the children of the table (contents)- This results in table body. Rows in the table are children of table_body <tbody>. To get all rows we call children of the first item in table body. Lets assign table_body to this code.

In [10]:
list_table.getchildren()

[<Element thead at 0x26d3f7f33b8>, <Element tbody at 0x26d3f7f34f8>]

There are 2 table elements - thead and tbody. thead has 1 row while tbody has 100 rows.

In [11]:
table_body = list_table.getchildren()[1]

In [12]:
len(table_body)

100

There are 100 total rows in the table body. The table head was downloaded as a separate element and is not to be removed. 

#Now we can pick the first row (index = 0) and get text content for that- that would show the text in first row. There are text related to the 4 Columns: Title, Release date, Publisher and Developer

In [13]:
table_body.getchildren()[0].text_content()

'1                              Tenuta San Guido Bolgheri-Sassicaia Sassicaia 2015 +-                                            Rich and concentrated, this red features black currant, blackberry, violet, mineral and spice flavors. Dense yet lively, structured yet impeccably balanced, with vibrant acidity driving the long, fruit-filled aftertaste. The oak is beautifully integrated. Cabernet Sauvignon and Cabernet Franc. Best from 2023 through 2042. 17,200 cases made. â\x80\x94BS                             Score 97 | Price $245                                            201597$245'

#Now we can pick the first row and get children for that- that would show the 4 columns in first row. These are table headers and therefore the result shows 'th' tags for each.


In [14]:
table_body.getchildren()[0].getchildren()

[<Element td at 0x26d3f7f39a8>,
 <Element td at 0x26d3f7f3598>,
 <Element td at 0x26d3f7f39f8>,
 <Element td at 0x26d3f7f3a48>,
 <Element td at 0x26d3f7f3a98>,
 <Element td at 0x26d3f7f3ae8>]

On the website, the first row in the table body 6 columns (Rank, Wine, Vintage, Score, Price and Selection). Lets check the second row and get children for that- that should also show the 6 columns. These result shows 'td' tags for each (table data?).

In [15]:
table_body.getchildren()[1].text_content()

"2                              ChÃ¢teau Canon-La GaffeliÃ¨re St.-Emilion 2015 +-                                            Still youthfully tight and backward, with a well-roasted frame of alder and juniper holding sway for now, but the core of cassis, blackberry and plum fruit waits in reserve, showing prodigious depth. When the toast and fruit melds, the backdrop of tobacco, singed iron and chalky minerality will get a turn to show. There's a lot here. Built for the cellar. Merlot, Cabernet Franc and Cabernet Sauvignon. Best from 2025 through 2040. 6,250 cases made. â\x80\x94JM                             Score 96 | Price $84                                            201596$84"

In [16]:
table_body.getchildren()[1].getchildren()

[<Element td at 0x26d3f7f3138>,
 <Element td at 0x26d3f7f3d18>,
 <Element td at 0x26d3f7f3d68>,
 <Element td at 0x26d3f7f3db8>,
 <Element td at 0x26d3f7f3e08>,
 <Element td at 0x26d3f7f3e58>]

Lets check the text content.

In [17]:
table_body.getchildren()[0].text_content()

'1                              Tenuta San Guido Bolgheri-Sassicaia Sassicaia 2015 +-                                            Rich and concentrated, this red features black currant, blackberry, violet, mineral and spice flavors. Dense yet lively, structured yet impeccably balanced, with vibrant acidity driving the long, fruit-filled aftertaste. The oak is beautifully integrated. Cabernet Sauvignon and Cabernet Franc. Best from 2023 through 2042. 17,200 cases made. â\x80\x94BS                             Score 97 | Price $245                                            201597$245'

In [18]:
table_body.getchildren()[0].getchildren()

[<Element td at 0x26d3f7f39a8>,
 <Element td at 0x26d3f7f3598>,
 <Element td at 0x26d3f7f39f8>,
 <Element td at 0x26d3f7f3a48>,
 <Element td at 0x26d3f7f3a98>,
 <Element td at 0x26d3f7f3ae8>]

By reading the text content for 1st row (as above) we can see all text in the row, but it also shows all the new line charecters. To see the data clearly, we can print the data in each cell as below:

In [19]:
print(table_body.getchildren()[0].getchildren()[0].text_content())
print(table_body.getchildren()[0].getchildren()[1].text_content())
print(table_body.getchildren()[0].getchildren()[2].text_content())
print(table_body.getchildren()[0].getchildren()[3].text_content())
print(table_body.getchildren()[0].getchildren()[4].text_content())
print(table_body.getchildren()[0].getchildren()[5].text_content())

1
                      Tenuta San Guido Bolgheri-Sassicaia Sassicaia 2015 +-                                            Rich and concentrated, this red features black currant, blackberry, violet, mineral and spice flavors. Dense yet lively, structured yet impeccably balanced, with vibrant acidity driving the long, fruit-filled aftertaste. The oak is beautifully integrated. Cabernet Sauvignon and Cabernet Franc. Best from 2023 through 2042. 17,200 cases made. âBS                             Score 97 | Price $245                                            
2015
97
$245



At this point we have scraped the table data from the website website and it is ready to be parsed using Pandas.
Lets define a pandas dataframe using a dictionary with keys corresponding to the table headers and empty lists to be filled in later using dynamic variables. The pandas dictionary will have 5 keys :Rank, Wine, Vintage, Score, Price.

In [20]:
data = {
    "Rank" : [],
    "Wine" : [],
    "Vintage" : [],
    "Score" : [],
    "Price" : []
}

Now that the dictinoary is setup, we loop over each row in the table_body starting from row 1 (index 0) using for loop and then append to the empty lists. Remove new line charecters using strip(). Try with 10 rows first.

In [21]:
for row in table_body.getchildren()[0:]:
    data["Rank"].append(row.getchildren()[0].text_content().strip())
    data["Wine"].append(row.getchildren()[1].text_content().strip())
    data["Vintage"].append(row.getchildren()[2].text_content().strip())
    data["Score"].append(row.getchildren()[3].text_content().strip())
    data["Price"].append(row.getchildren()[4].text_content().strip())    

In [22]:
df = pd.DataFrame(data)

In [23]:
df

,Rank,Wine,Vintage,Score,Price
0,1,Tenuta San Guido Bolgheri-Sassicaia Sassicaia ...,2015,97,$245
1,2,ChÃ¢teau Canon-La GaffeliÃ¨re St.-Emilion 2015...,2015,96,$84
2,3,Castello di Volpaia Chianti Classico Riserva 2...,2015,96,$35
3,4,La Rioja Alta Rioja 890 Gran Reserva SelecciÃ³...,2005,95,$175
4,5,MoÃ«t & Chandon Brut Champagne Dom PÃ©rignon L...,2008,96,$180
5,6,Aubert Chardonnay Carneros Larry Hyde & Sons 2...,2016,96,$85
6,7,Colene Clemens Pinot Noir Chehalem Mountains D...,2015,95,$26
7,8,Le Vieux Donjon ChÃ¢teauneuf-du-Pape 2016 +- ...,2016,95,$70
8,9,Tenuta delle Terre Nere Etna San Lorenzo 2016 ...,2016,95,$60
9,10,Bedrock The Bedrock Heritage Sonoma Valley 201...,2016,95,$46


Since no errors occured, go back to the code and extend the code to all data / rows in the table.

In [24]:
len(df)

100

This data frame (df) has 100 rows and is now ready for further explore data using pandas or to be loaded in to the sqlite3 database.

In [25]:
trial = df[df.Vintage.isin(["2017", "2016"])]

In [26]:
trial

,Rank,Wine,Vintage,Score,Price
5,6,Aubert Chardonnay Carneros Larry Hyde & Sons 2...,2016,96,$85
7,8,Le Vieux Donjon ChÃ¢teauneuf-du-Pape 2016 +- ...,2016,95,$70
8,9,Tenuta delle Terre Nere Etna San Lorenzo 2016 ...,2016,95,$60
9,10,Bedrock The Bedrock Heritage Sonoma Valley 201...,2016,95,$46
11,12,Felton Road Pinot Noir Central Otago Bannockbu...,2017,96,$50
13,14,Warre Vintage Port 2016 +- ...,2016,98,$98
18,19,San Felice Chianti Classico 2016 +- ...,2016,94,$17
20,21,Scott Base Pinot Noir Central Otago 2016 +- ...,2016,93,$18
22,23,Taylor Fladgate Vintage Port 2016 +- ...,2016,98,$120
24,25,Tensley Syrah Santa Barbara County Colson Cany...,2016,94,$42


In [27]:
len(trial)

49

There are 49 wines in the top 100 that are vintage 2016 or 2017.

In [28]:
trial = df[df.Price.isin(["$13", "$15"])]

In [29]:
trial

,Rank,Wine,Vintage,Score,Price
39,40,Cline Zinfandel Contra Costa County Ancient Vi...,2016,91,$15
46,47,EdetÃ ria Garnatxa Blanca Terra Alta Via Terra...,2017,91,$15
55,56,Yalumba Viognier South Australia The Y Series ...,2017,90,$13
61,62,Joel Gott Sauvignon Blanc California 2017 +- ...,2017,90,$15
63,64,Terra Santa Ãle de BeautÃ© White 2017 +- ...,2017,90,$15
80,81,Librandi Val di Neto White Critone 2017 +- ...,2017,90,$15
92,93,Carol Shelton RosÃ© Wild Thing Rendezvous Mend...,2017,90,$15


Let us load the new data frame trial in to the sqlite3 database.

In [30]:
len(trial)

7

There are 7 wines in the top 100 that cost $15 or less.

The database operation and data can either be stored as a file in the computer by defining a file name or be stored in the process memory to be made available as long as the Notbook kernel is running. This is temporary and requires special syntax :  db = sqlite3.connect(":memory:").
In this example, we will save the database on the computer.

In [31]:
conn = sqlite3.connect('webscraping.db')
c = conn.cursor()

Lets create a new table called Wine in the database webscraping.db

In [32]:
c.execute("""
    CREATE TABLE IF NOT EXISTS BestWine(Rank, Wine, Vintage, Score, Price)
""")

In [34]:
for row in trial.itertuples():
    insert_table = """
    INSERT INTO BestWine(Rank, Wine, Vintage, Score, Price) \
    VALUES (?,?,?,?,?)
"""
    c.execute(insert_table, row[1:6])
conn.commit()

A new database called webscraping.db has been created with a new table called BestWine with 7 rows of data.
We can now retrieve data from the new database.
Lets query for all Wines that are priced at $15

In [35]:
for row in c.execute("""
    SELECT *
    FROM BestWine
    WHERE Price = "$15"
"""):
    print(row)

('40', 'Cline Zinfandel Contra Costa County Ancient Vines 2016 +-                                            Lively and supple, with floral raspberry and fresh pepper aromas and sleekly appealing cherry and licorice flavors that glide lightly along the finish. Drink now through 2022. 50,000 cases made. â\x80\x94TF                             Score 91 | Price $15', '2016', '91', '$15')
('47', 'EdetÃ\xa0ria Garnatxa Blanca Terra Alta Via Terra 2017 +-                                            This white has a thick texture, delivering pear, guava, tarragon and blanched almond flavors. Firm acidity and a touch of tannins give this a solid frame. Muscular, but remains balanced. Drink now through 2023. 5,000 cases made. â\x80\x94TM                             Score 91 | Price $15', '2017', '91', '$15')
('62', 'Joel Gott Sauvignon Blanc California 2017 +-                                            Juicy pear, peach and citrus flavors are bright and smooth, with a touch of dried chamomile an

Lets query for all Wines priced at $15 or under which have a vintage of 2017

In [36]:
for row in c.execute("""
    SELECT *
    FROM BestWine
    WHERE Vintage = "2017"
"""):
    print(row)

('47', 'EdetÃ\xa0ria Garnatxa Blanca Terra Alta Via Terra 2017 +-                                            This white has a thick texture, delivering pear, guava, tarragon and blanched almond flavors. Firm acidity and a touch of tannins give this a solid frame. Muscular, but remains balanced. Drink now through 2023. 5,000 cases made. â\x80\x94TM                             Score 91 | Price $15', '2017', '91', '$15')
('56', 'Yalumba Viognier South Australia The Y Series 2017 +-                                            Lemon, chamomile and spicy nectarine flavors are fresh and vibrant, set on a crisp body, finishing spicy and refreshing, with plenty of intensity. Drink now. 19,000 cases imported. â\x80\x94MW                             Score 90 | Price $13', '2017', '90', '$13')
('62', 'Joel Gott Sauvignon Blanc California 2017 +-                                            Juicy pear, peach and citrus flavors are bright and smooth, with a touch of dried chamomile and lemon verbena li

Lets make the data more readable, by printing each element per row and using for loop

In [37]:
data = c.execute("""SELECT * 
                    FROM BestWine 
                    WHERE Vintage= "2017"
""")
for row in data:
   print("Rank = ", row[0])
   print("Wine = ", row[1])
   print("Vintage = ", row[2])
   print("Score = ", row[3])
   print("Price = ", row[4],"\n")
conn.close()

Rank =  47
Wine =  EdetÃ ria Garnatxa Blanca Terra Alta Via Terra 2017 +-                                            This white has a thick texture, delivering pear, guava, tarragon and blanched almond flavors. Firm acidity and a touch of tannins give this a solid frame. Muscular, but remains balanced. Drink now through 2023. 5,000 cases made. âTM                             Score 91 | Price $15
Vintage =  2017
Score =  91
Price =  $15 

Rank =  56
Wine =  Yalumba Viognier South Australia The Y Series 2017 +-                                            Lemon, chamomile and spicy nectarine flavors are fresh and vibrant, set on a crisp body, finishing spicy and refreshing, with plenty of intensity. Drink now. 19,000 cases imported. âMW                             Score 90 | Price $13
Vintage =  2017
Score =  90
Price =  $13 

Rank =  62
Wine =  Joel Gott Sauvignon Blanc California 2017 +-                                            Juicy pear, peach and citrus flavors are bright and sm